In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from dateutil.parser import parse

import os
import sys
import re

In [2]:
# Set file path
file_path = '/Users/jiminking/Desktop/LA/WebScraping/coupang_baby.csv'

# Load data to pandas DataFrame
data = pd.read_csv(file_path)
scraping_date = datetime.datetime(2024, 1, 18)

# Data info
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Item Name       1020 non-null   object
 1   Discount        1020 non-null   object
 2   Original Price  1020 non-null   object
 3   Sale Price      1020 non-null   object
 4   Delivery Info   1018 non-null   object
 5   Rating Score    1020 non-null   object
 6   Total Reviews   1020 non-null   object
dtypes: object(7)
memory usage: 55.9+ KB
None


,Item Name,Discount,Original Price,Sale Price,Delivery Info,Rating Score,Total Reviews
0,"주니 유아용 자기주도 빨대컵 210ml, 화이트, 1개",28%,"35,000","25,000",내일(금) \n 도착 보장,90,119
1,"한글떼기 1~5과정 세트 전5권(개정판)(유아4~7세), 기탄출판",10%,"30,000","27,000",내일(금) \n 도착 보장,100,1692
2,"초등 미니 논술 일력 365, 서사원주니어",10%,"19,800","17,820",내일(금) \n 도착 보장,100,18
3,"락피도 철분 키즈, 90g, 1개",55%,"37,000","16,540",내일(금) \n 도착 보장,100,2905
4,"아토팜 판테놀 유아 크림, 100ml, 1개",48%,"47,000","24,110",내일(금) \n 도착 보장,100,740


In [3]:
def preprocess_column_name(df):
    """
    Preprocesses column names.

    Args:
        df (pandas DataFrame): The DataFrame to preprocess.

    Returns:
        pandas DataFrame: The DataFrame with preprocessed column names.
    """

    df.columns = df.columns.str.lower()  # Convert column names to lowercase
    df.columns = df.columns.str.replace(" ", "_")  # Replace spaces with underscores
    df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
    return df

def split_item_name(item_name):
    """
    Splits the item name by comma and returns the first part.

    Args:
        item_name (str): The item name to split.

    Returns:
        str: The first part of the item name before the comma.
    """

    names = item_name.split(',')
    return names[0]

def process_item_info(item_name):
    """
    Processes item info.

    Args:
        item_name (str): The item name to process.

    Returns:
        str: A comma-separated string of item information after the first comma,
              or an empty string if there's no additional information.
    """

    split_items = item_name.split(',')
    return ','.join(split_items[1:]) if len(split_items) > 1 else ''

def preprocess_price(df):
    """
    Preprocesses price columns by removing commas and converting to numeric values.

    Args:
        df (pandas DataFrame): The DataFrame to preprocess.

    Returns:
        pandas DataFrame: The DataFrame with preprocessed price columns.
    """

    for col in ['original_price', 'sale_price']:
        df[col] = df[col].str.replace(',', '')  # Remove commas
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric
        df[col] = df[col].astype('Int64')  # Convert to Int64 type
    return df

def preprocess_delivery_info(delivery_info):
    """
    Preprocesses delivery information by removing newlines and extra spaces.

    Args:
        delivery_info (str): The delivery information to preprocess.

    Returns:
        str: The preprocessed delivery information.
    """

    if pd.isna(delivery_info):
        return delivery_info
    return ' '.join(delivery_info.replace('\n', '').split())

def determine_delivery_certainty(delivery_info):
    """
    Determines delivery certainty based on delivery information.

    Args:
        delivery_info (str): The delivery information to analyze.

    Returns:
        str: The delivery certainty ('확실', '미확실', or '정보 없음').
    """

    if pd.isna(delivery_info):
        return '정보 없음'
    elif '보장' in delivery_info:
        return '확실'
    elif '예정' in delivery_info:
        return '미확실'
    else:
        return '정보 없음'

def preprocess_delivery_guarantee(df):
    """
    Preprocesses the 'delivery_guarantee' column by encoding '확실' as 1 and '미확실' as 0.

    Args:
        df (pandas DataFrame): The DataFrame to preprocess.

    Returns:
        pandas DataFrame: The DataFrame with the preprocessed 'delivery_guarantee' column.
    """

    mapping = {'확실': 1, '미확실': 0}
    df['delivery_guarantee'] = df['delivery_guarantee'].map(mapping)
    return df

def calculate_delivery_days(delivery_info):
# 날짜 정보를 파싱하고, 배송 기간을 계산하는 함수
    try:
        if '내일' in delivery_info:
            return 1  # tomorrow
        elif '모레' in delivery_info:
            return 2  # after 2 days
        elif 'No delivery info' in delivery_info:
            return None  # no delivery info
        else:
            # date parsing (예: "1/22(월)")
            delivery_date = parse(delivery_info.split()[0], dayfirst=False, yearfirst=False)
            # set 2024 (년도 정보가 없기 때문에)
            delivery_date = delivery_date.replace(year=2024)
            # calculate the number of days until delivery
            return (delivery_date - scraping_date).days
    except Exception as e:
        return None  # error handling
    
def make_csv_file(df,output_file_path):
    """
    save dataframe to csv file
    Args : 
        df : dataframe
        output_file_path : output file path
    Return : 
        None
    """
    df.to_csv(output_file_path, index=False, encoding='UTF-8-sig')
    print(f"{output_file_path} is created successfully")


In [4]:
print("====================Before the preprocessing====================")
print(data.info())
print(data.shape)


print("====================After the preprocessing====================")
data = preprocess_column_name(data)
data["item_name_"] = data["item_name"].apply(split_item_name)
data["item_info"] = data["item_name"].apply(process_item_info)
data = data.drop(columns=["item_name"])
data = preprocess_price(data)
data['discount_percentage'] = data['discount'].replace('No discount', '0%').str.replace('%', '').astype(int)
data['delivery_info'] = data['delivery_info'].apply(preprocess_delivery_info)
data['delivery_guarantee'] = data['delivery_info'].apply(determine_delivery_certainty)
data['delivery_days'] = data['delivery_info'].apply(calculate_delivery_days)
data = data.drop(columns=["delivery_info"])
data = preprocess_delivery_guarantee(data)

print(data.info())
data.head()

====================Before the preprocessing====================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Item Name       1020 non-null   object
 1   Discount        1020 non-null   object
 2   Original Price  1020 non-null   object
 3   Sale Price      1020 non-null   object
 4   Delivery Info   1018 non-null   object
 5   Rating Score    1020 non-null   object
 6   Total Reviews   1020 non-null   object
dtypes: object(7)
memory usage: 55.9+ KB
None
(1020, 7)
====================After the preprocessing====================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   discount             1020 non-null   object 
 1   original_price       796 non-null    Int64  
 2   s

,discount,original_price,sale_price,rating_score,total_reviews,item_name_,item_info,discount_percentage,delivery_guarantee,delivery_days
0,28%,35000,25000,90,119,주니 유아용 자기주도 빨대컵 210ml,"화이트, 1개",28,1.0,1.0
1,10%,30000,27000,100,1692,한글떼기 1~5과정 세트 전5권(개정판)(유아4~7세),기탄출판,10,1.0,1.0
2,10%,19800,17820,100,18,초등 미니 논술 일력 365,서사원주니어,10,1.0,1.0
3,55%,37000,16540,100,2905,락피도 철분 키즈,"90g, 1개",55,1.0,1.0
4,48%,47000,24110,100,740,아토팜 판테놀 유아 크림,"100ml, 1개",48,1.0,1.0


In [5]:
# Reordering the dataframe to have 'item_name_' column at the front
cols = list(data.columns)
if 'item_name_' in cols:
    cols.insert(0, cols.pop(cols.index('item_name_')))
    data_reordered = data[cols]
else:
    data_reordered = data # No need to reorder if 'item_name_' column doesn't exist


In [6]:
# Removing rows with NaN values in the data_reordered dataframe
data_cleaned = data_reordered.dropna()
data_cleaned['delivery_guarantee'] = data_cleaned['delivery_guarantee'].astype(int)
data_cleaned['delivery_days'] = data_cleaned['delivery_days'].astype(int)


# Displaying the updated dataframe
data_cleaned

/var/folders/mq/s4hsb4k169q9b9n3njn70_sh0000gn/T/ipykernel_13264/1841610438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['delivery_guarantee'] = data_cleaned['delivery_guarantee'].astype(int)
/var/folders/mq/s4hsb4k169q9b9n3njn70_sh0000gn/T/ipykernel_13264/1841610438.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['delivery_days'] = data_cleaned['delivery_days'].astype(int)


,item_name_,discount,original_price,sale_price,rating_score,total_reviews,item_info,discount_percentage,delivery_guarantee,delivery_days
0,주니 유아용 자기주도 빨대컵 210ml,28%,35000,25000,90,119,"화이트, 1개",28,1,1
1,한글떼기 1~5과정 세트 전5권(개정판)(유아4~7세),10%,30000,27000,100,1692,기탄출판,10,1,1
2,초등 미니 논술 일력 365,10%,19800,17820,100,18,서사원주니어,10,1,1
3,락피도 철분 키즈,55%,37000,16540,100,2905,"90g, 1개",55,1,1
4,아토팜 판테놀 유아 크림,48%,47000,24110,100,740,"100ml, 1개",48,1,1
...,...,...,...,...,...,...,...,...,...,...
1015,랄랄라 사운드 벽보 8: 구구단,37%,5000,3150,90,5078,키움,37,1,1
1016,종이나라 투명나라풀,53%,14400,6630,90,7445,"35g, 12개",53,1,1
1017,지앤마 써니비 남아용 1단계 유아칫솔,13%,9900,8550,100,398,"1개입, 20개",13,1,1
1018,2080 브레드이발소 키즈 치약 3스텝 베리믹스향,57%,18900,8000,100,3662,"80g, 6개",57,1,1


In [7]:
make_csv_file(data_cleaned, '/Users/jiminking/Desktop/LA/WebScraping/coupang_baby_preprocessed.csv')

/Users/jiminking/Desktop/LA/WebScraping/coupang_baby_preprocessed.csv is created successfully
